### Sharpening_with_OpenCv

In [ ]:
import cv2 
import numpy as np 
 
img = cv2.imread('02.jpg') 
img = cv2.resize(img, None, fx=0.25, fy=0.25,interpolation=cv2.INTER_CUBIC)
cv2.imshow('Original', img) 
 
# generating the kernels 
kernel_sharpen_1 = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) 
kernel_sharpen_2 = np.array([[1,1,1], [1,-7,1], [1,1,1]]) 
kernel_sharpen_3 = np.array([[-1,-1,-1,-1,-1], 
                             [-1,2,2,2,-1], 
                             [-1,2,8,2,-1], 
                             [-1,2,2,2,-1], 
                             [-1,-1,-1,-1,-1]]) / 8.0 
 
# applying different kernels to the input image 
output_1 = cv2.filter2D(img, -1, kernel_sharpen_1) 
output_2 = cv2.filter2D(img, -1, kernel_sharpen_2) 
output_3 = cv2.filter2D(img, -1, kernel_sharpen_3) 
 
cv2.imshow('Sharpening', output_1) 
cv2.imshow('Excessive Sharpening', output_2) 
cv2.imshow('Edge Enhancement', output_3) 
cv2.waitKey(0) 


### Make merge intersection of two Image using bitwise

In [ ]:

# Load two images
img1 = cv2.imread('bongSung.jpg')
img2 = cv2.imread('opencv.jpg')

# I want to put logo on top-left corner, so I create a ROI 
rows, cols, channels = img2.shape
roi = img1[0:rows, 0:cols]

# NOw we need to create a mask of the logo, mask is conversion to grayscale of an image
img2gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) 
ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('mask', mask)

mask_inv = cv2.bitwise_not(mask)
#cv2.imshow("mask_inv", mask_inv)

#When using bitwise_and() in opencv with python then white + anycolor = anycolor; black + anycolor = black 
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
#cv2.imshow("img1_bg", img1_bg)

cv2.imshow("img2", img2)

img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
cv2.imshow('img2_fg', img2_fg)

dst = cv2.add(img1_bg,img2_fg)

img1[0:rows, 0:cols] = dst

#cv2.imshow("Image", img1)
cv2.waitKey(0)

cv2.destroyAllWindows()

### Make the dilation, erosion, closing and Opening of some image

https://www.ft.unicamp.br/docentes/magic/khoros/html-dip/c9/s4/front-page.html

In [ ]:
import cv2 
import numpy as np 
 
img = cv2.imread('02.jpg', 0)
img = cv2.resize(img, None, fx=0.25, fy=0.25,interpolation=cv2.INTER_CUBIC)
 
kernel = np.ones((5,5), np.uint8) 
 
img_erosion = cv2.erode(img, kernel, iterations=1) 
img_dilation = cv2.dilate(img, kernel, iterations=1)

### Abertura= erosão seguida por dilatação
img_abertura = cv2.dilate(img_erosion,kernel, iterations=1)

### Fechamento = dilatação seguida por uma operação de erosão
img_fechamento = cv2.erode(img_dilation,kernel,iterations=1)

###  Suavilição não linear Abertura seguindo de Fechamento
sua_1 = cv2.dilate(img_abertura, kernel, iterations=1)
sua_2 = cv2.erode(sua_1,kernel,iterations=1)


cv2.imshow('Input', img) 
cv2.imshow('Erosion', img_erosion) 
cv2.imshow('Dilation', img_dilation) 
cv2.imshow('Abertura = E(D)', img_abertura) 
cv2.imshow('Fechamento= D(E)', img_fechamento) 
cv2.imshow('Suavilição Não linear ', sua_2) 
 
cv2.waitKey(0)

### Watershed_Algoritms

In [ ]:
import numpy as np
import cv2
import sys


if __name__=='__main__':
    # Input image containing all the different shapes
    img = cv2.imread(sys.argv[1])
    img = cv2.resize(img, None, fx=0.25, fy=0.25,interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=4)
    # sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    markers = cv2.watershed(img, markers)
    img[markers==-1] = [255, 255, 255]
    cv2.imshow('background', sure_bg)
    cv2.imshow('foreground', sure_fg)
    cv2.imshow('threshold', thresh)
    cv2.imshow('result', img)
    cv2.waitKey(0)
